# Cleaning the Source Data Set

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport
from pathlib import Path

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [4]:
df = pd.read_csv("1.a.Detail_Incident.csv", parse_dates=['Open_Time', 'Reopen_Time', 'Resolved_Time','Close_Time', ])

In [5]:
df.dtypes

CI_Name_aff                           object
CI_Type_aff                           object
CI_Subtype_aff                        object
Service_Component_WBS_aff             object
Incident_ID                           object
Status                                object
Impact                                 int64
Urgency                                int64
Priority                               int64
Category                              object
KM_number                             object
Alert_Status                          object
Count_Reassignments                  float64
Open_Time                     datetime64[ns]
Reopen_Time                   datetime64[ns]
Resolved_Time                 datetime64[ns]
Close_Time                    datetime64[ns]
Handle_Time_Hours                    float64
Closure_Code                          object
Count_Related_Interactions           float64
Related_Interaction                   object
Count_Related_Incidents              float64
Count_Rela

## Drop Records where Resolved_Time is Missing

In [6]:
df.iloc[:,13:17].isnull().sum()

Open_Time            0
Reopen_Time      44322
Resolved_Time     1780
Close_Time           0
dtype: int64

In [7]:
df = df.dropna(subset=['Resolved_Time'])

In [8]:
df.iloc[:,13:17].isnull().sum()

Open_Time            0
Reopen_Time      42607
Resolved_Time        0
Close_Time           0
dtype: int64

## Limit timeframe of all records

greater than 1 october 2013

less than 31 march 2014


In [9]:
df = df[df['Open_Time'] >= pd.to_datetime('10-01-2013')]

In [10]:
df.iloc[:,13:17].describe()

,Open_Time,Reopen_Time,Resolved_Time,Close_Time
count,43709,2038,43709,43709
unique,43455,2036,43496,43500
top,2014-01-22 15:46:06,2013-11-12 10:36:33,2013-11-22 16:34:33,2014-02-27 15:04:32
freq,3,2,3,3
first,2013-10-01 07:33:21,2013-10-01 11:43:47,2013-10-01 08:18:27,2013-10-01 08:18:30
last,2014-03-31 17:24:49,2014-03-31 16:21:15,2014-03-31 22:47:29,2014-03-31 22:47:32


## Deal with Status of 'work in progress'

In [11]:
df.Status.value_counts()

Closed              43700
Work in progress        9
Name: Status, dtype: int64

In [12]:
df = df[ df['Status'] == 'Closed' ]

In [13]:
df.Status.value_counts()

Closed    43700
Name: Status, dtype: int64

## Remove non-incident records

In [14]:
print(df.Category.value_counts())

incident                   35208
request for information     8482
complaint                      9
request for change             1
Name: Category, dtype: int64


In [15]:
df = df[ df['Category'] == 'incident' ]

In [16]:
print(df.Category.value_counts())
print(df.Status.value_counts())
print(df.Alert_Status.value_counts())

incident    35208
Name: Category, dtype: int64
Closed    35208
Name: Status, dtype: int64
closed    35208
Name: Alert_Status, dtype: int64


## Deal with Reopen_Time Missing Values


In [17]:
df.Reopen_Time.isnull().sum()

33782

In [18]:
df['ReopenedFlag'] = ~ df.Reopen_Time.isnull()

In [19]:
df['ReopenedFlag'] = df['ReopenedFlag'].astype(int)

In [20]:
df['ReopenedFlag'].value_counts()

0    33782
1     1426
Name: ReopenedFlag, dtype: int64

## Set Missing to Zero for `Count_Related_Changes`, `Count_Related_Incidents`, and `Count_Related_Interactions` 

In [22]:
print(df['Count_Related_Changes'].isnull().sum())
print(df['Count_Related_Incidents'].isnull().sum())
print(df['Count_Related_Interactions'].isnull().sum())


34732
34164
111


In [23]:
df['Count_Related_Changes'] = df['Count_Related_Changes'].fillna(0)
df['Count_Related_Incidents'] = df['Count_Related_Incidents'].fillna(0)
df['Count_Related_Interactions'] = df['Count_Related_Interactions'].fillna(0)


In [24]:
print(df['Count_Related_Changes'].isnull().sum())
print(df['Count_Related_Incidents'].isnull().sum())
print(df['Count_Related_Interactions'].isnull().sum())

0
0
0


## Set Missing to "Not Applicable" for `Related_Change`

In [25]:
df['Related_Change'].value_counts().sum()

476

In [26]:
df['Related_Change'] = df['Related_Change'].fillna("Not Applicable")

In [27]:
df['Related_Change'].value_counts()

Not Applicable    34732
C00003013           110
C00014762            78
#MULTIVALUE          18
C00001012            10
C00012714            10
C00000713             9
C00009165             7
C00009722             7
C00008750             5
C00017302             5
C00014221             5
C00006833             4
C00001807             3
C00015613             3
C00004344             3
C00001026             3
C00000829             3
C00009821             3
C00012116             2
C00008442             2
C00013454             2
C00000527             2
C00013072             2
C00003404             2
C00016781             2
C00014458             2
C00002268             2
C00006448             2
C00001507             2
C00012545             2
C00008222             2
C00004294             2
C00007098             2
C00001250             2
C00008726             2
C00002389             2
C00016192             2
C00004739             2
C00005261             2
C00001549             2
C00005866       

## Drop columns 

with constant values,   

and no longer needed, `Reopen_Time`


In [28]:
df = df.drop(['Category', 'Status', 'Alert_Status', 'Reopen_Time'], axis='columns')

In [29]:
df.columns

Index(['CI_Name_aff', 'CI_Type_aff', 'CI_Subtype_aff',
       'Service_Component_WBS_aff', 'Incident_ID', 'Impact', 'Urgency',
       'Priority', 'KM_number', 'Count_Reassignments', 'Open_Time',
       'Resolved_Time', 'Close_Time', 'Handle_Time_Hours', 'Closure_Code',
       'Count_Related_Interactions', 'Related_Interaction',
       'Count_Related_Incidents', 'Count_Related_Changes', 'Related_Change',
       'CI_Name_CBy', 'CI_Type_CBy', 'CI_Subtype_CBy', 'ServiceComp_WBS_CBy',
       'ReopenedFlag'],
      dtype='object')

## END and OUTPUT

In [30]:
with open("2.a.Detail_Incident.csv",'w') as f:
    df.to_csv(f, index=False)

In [31]:
df.reset_index(drop=True, inplace=True)
profile = ProfileReport(df, title="Profile of BPIC 2014 Detail_Incident Data after Secondary Cleaning", html={'style': {'full_width': True}})

In [32]:
profile.to_file(Path(str("2.b.Detail_Incident_Profile.html")))